## Laboratory 2 - LLM

### HES-SO MSE - Web Mining

This file will guid you through the exercises of the laboratory. All questions are marqued in <span style="color:red">**red**</span>. Please answer them with the empty cell below as there's no report to give. The answer can be python code (*feel free to add comments !*) or simply text.

### Exercise 1 - Meet Ollama !

Let's start by using Ollama, a module called [ollama-python](https://github.com/ollama/ollama-python) will help us to make simple tasks with the `LLaMA` model. Basically this library will simplify the requests made to the API created with our docker container (or server depending on how you installed it). 

In [ ]:
import ollama
import numpy as np

<span style="color:red">1.1 Start by downloading the model `llama2`</span>

In [ ]:
ollama.pull('llama2')

<span style="color:red">1.2 How many parameters does this model has ?</span>

7B

<span style="color:red">1.3 Ask him any question, feel free to ask what you want.</span>

In [ ]:
response = ollama.chat(model='llama2', messages=[
  {
    'role': 'user',
    'content': 'Hello comment tu vas',
  },
])
print(response['message']['content'])

<span style="color:red">1.4 Simulate a conversation with the chat bot. Use only one call and "fake" atleast one message from the bot too.</span>

In [ ]:
response = ollama.chat(model='llama2', messages=[
  {
    'role': 'user',
    'content': 'Hello, combien fort 6 * 2',
  },
  {
    'role': 'assistant',
    'content': '12',
  },
  {
    'role': 'user',
    'content': 'Combien font 2+2 + la réponse précédente',
  }
])

print(response['message']['content'])


<span style="color:red">1.5 Create a new model that uses a `Modelfile`. Tell in the ModelFile that the bot needs to show that it is a robot. Change parameters so the model is more creative. Then try it by chatting with it.</span>

In [ ]:
modelfile='''
FROM llama2
SYSTEM You are mario from super mario bros.
'''

ollama.create(model='example', modelfile=modelfile)



response = ollama.chat(model='example', messages=[
  {
    'role': 'user',
    'content': 'Tu fais quoi dans la vie',
  },
])
print(response['message']['content'])


<span style="color:red">1.6 Delete the model created</span>

In [ ]:
ollama.delete('example')

<span style="color:red">1.7 Use the `main_prompt` variable to compare its cosine similarities with the other paragraphs. Fill the `cosine_sim` function without using any other libraries other than `numpy`. Then compute their similarities by using it.</span>

*PS: These sentences come from the first paragraph of Wikipedia of the following subjects :*
* *Switzerland* (`main_prompt`)
* *France* (`prompts[0]`)
* *Chocolate* (`prompts[1]`)
* *Europe* (`prompts[2]`)
* *e (mathematical constant)* (`prompts[3]`)
* *Chewing gum* (`prompts[4]`)

In [ ]:
import numpy as np
main_prompt = "Switzerland, officially the Swiss Confederation, is a landlocked country located in west-central Europe. It is bordered by Italy to the south, France to the west, Germany to the north and Austria and Liechtenstein to the east. Switzerland is geographically divided among the Swiss Plateau, the Alps and the Jura; the Alps occupy the greater part of the territory, whereas most of the country's population of 9 million are concentrated on the plateau, which hosts the largest cities and economic centres, including Zürich, Geneva and Basel."
prompts = [
    "France, officially the French Republic (French: République française is a country located primarily in Western Europe. It also includes overseas regions and territories in the Americas and the Atlantic, Pacific and Indian oceans, giving it one of the largest discontiguous exclusive economic zones in the world. Metropolitan France shares borders with Belgium and Luxembourg to the north, Germany to the north east, Switzerland to the east, Italy and Monaco to the south east, Andorra and Spain to the south, and a maritime border with the United Kingdom to the north west. Its metropolitan area extends from the Rhine to the Atlantic Ocean and from the Mediterranean Sea to the English Channel and the North Sea.",
    "A computer is a machine that can be programmed to carry out sequences of arithmetic or logical operations (computation) automatically. Modern digital electronic computers can perform generic sets of operations known as programs. These programs enable computers to perform a wide range of tasks. The term computer system may refer to a nominally complete computer that includes the hardware, operating system, software, and peripheral equipment needed and used for full operation; or to a group of computers that are linked and function together, such as a computer network or computer cluster.",
    "Chocolate or cocoa is a food made from roasted and ground cacao seed kernels that is available as a liquid, solid, or paste, either on its own or as a flavoring agent in other foods. Cacao has been consumed in some form for at least 5,300 years starting with the Mayo-Chinchipe culture in what is present-day Ecuador and later Mesoamerican civilizations also consumed chocolate beverages before being introduced to Europe in the 16th century.",
    "Europe is a continent located entirely in the Northern Hemisphere and mostly in the Eastern Hemisphere. It is bordered by the Arctic Ocean to the north, the Atlantic Ocean to the west, the Mediterranean Sea to the south, and Asia to the east. Europe shares the landmass of Eurasia with Asia, and of Afro-Eurasia with both Asia and Africa. Europe is commonly considered to be separated from Asia by the watershed of the Ural Mountains, the Ural River, the Caspian Sea, the Greater Caucasus, the Black Sea, and the waterways of the Turkish straits.",
    "The number e is a mathematical constant approximately equal to 2.71828 that can be characterized in many ways. It is the base of natural logarithms. It is the limit of (1 + 1/n)n as n approaches infinity, an expression that arises in the computation of compound interest. It can also be calculated as the sum of the infinite series.",
    "Chewing gum is a soft, cohesive substance designed to be chewed without being swallowed. Modern chewing gum is composed of gum base, sweeteners, softeners/plasticizers, flavors, colors, and, typically, a hard or powdered polyol coating.[1] Its texture is reminiscent of rubber because of the physical-chemical properties of its polymer, plasticizer, and resin components, which contribute to its elastic-plastic, sticky, chewy characteristics.",
]



main_emb = ollama.embeddings(model='llama2', prompt=main_prompt)
# same for all prompts
prompt_embs = [ollama.embeddings(model='llama2', prompt=prompt) for prompt in prompts]

def cosine_sim(a, b):
    dot_product = np.dot(a['embedding'], b['embedding'])
    norm_a = np.linalg.norm(a['embedding'])
    norm_b = np.linalg.norm(b['embedding'])
    similarity = dot_product / (norm_a * norm_b)
    return similarity


# cosine similarity
cosine_similarities = [cosine_sim(main_emb, prompt_emb) for prompt_emb in prompt_embs]

print(cosine_similarities)



<span style="color:red">1.8 Explain what these embedding vectors represent and how are they obtained. Then explain the reasons of your results from the exercise 1.7.</span>

#Todo

### Exercise 2 - Retrieval Augmented Generation (RAG)


For this section, we're using the Open-Source framework **[LangChain](https://github.com/langchain-ai/langchain)**, which provides us a wide range of LLM-related functions.

In [7]:
from langchain_community import llms
from langchain import hub

<span style='color:red'>2.1 Connect to Ollama through the LangChain Framework and try to ask him about something more recent than July 2023.</span>

*If you don't know what to ask, try to talk about Gemma the new LLM Open-Source from Google.*

In [13]:
llm = llms.Ollama(model="llama2")

response = llm.invoke('what do you know about Gemma, the new open-source language model from Google?')
print(response)


Gemma is a newly announced open-source language model developed by Google. Here are some key things we know about it so far:

1. Gemma is a transformer-based language model: Like many other state-of-the-art language models, Gemma is based on the transformer architecture, which is particularly well-suited for natural language processing tasks.
2. It's trained on a large corpus of text: Google has trained Gemma on a massive dataset of text from various sources, including books, articles, and websites. This allows the model to learn patterns and relationships in language at scale.
3. It's designed for conversational AI: Gemma is intended for use in conversational AI applications, such as chatbots, voice assistants, and other dialogue systems. It can generate responses to user input in a way that's natural and contextually appropriate.
4. It's open-source: Google has released Gemma under an open-source license, which allows developers and researchers to access the model's source code, mod

<span style='color:red'>2.2 It's possible to have a positive answer (that seems correct) or a negative answer (doesn't know about it). Can you explain why these two answers are possible ?</span>

According to [Hugging Face](https://huggingface.co/meta-llama/Llama-2-7b), the model was trained between january 2023 and july 2023. Any information published after this date will not be known by the model.
However, the model has some genealization capabilities and can gives answers based on similar topics.
This inference can explain why the model can give a positive answer.
If the query is too specific or the topic is too recent, the model will not be able to give a correct answer and will give a negative answer.

<span style='color:red'>2.3 Create a pipeline that uses a [WebBaseLoader](https://python.langchain.com/docs/integrations/document_loaders/web_base) that reads a Web page that contains the answer to your question from 2.1. Then use the `rlm/rag-prompt` prompt for the RAG. Finally, ask the same question.</span>

*Help 1: You need to transform your documents into embedding vectors. For this, use `GPT4AllEmbeddings` instead of `OllamaEmbedding` because otherwise it can be really slow.*

*Help 2: You might need some other imports.*

*Help 3: This cell can run for a long time, don't take a website too big !*

*If you asked about Gemma, use [this blog page](https://blog.google/technology/developers/gemma-open-models/).*

In [39]:
from langchain.chains import RetrievalQA
from langchain.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import VectorStoreRetriever

# Load with WebBaseLoader
#TODO
loader = WebBaseLoader('https://blog.google/technology/developers/gemma-open-models/')
data = loader.load()

# Split & Store
#TODO
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

# RAG prompt
#TODO
prompt = hub.pull("rlm/rag-prompt")
llm = llms.Ollama(model="llama2")

# RetrievalQA to link the prompt to the vector store
#TODO

# # Ask the question
question = "what do you know about Gemma, the new open-source language model from Google?"

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectorstore.as_retriever(), chain_type_kwargs={"prompt": prompt}
)

answer = qa_chain({"query": question})
print(answer['result'])


Gemma is a new open-source language model introduced by Google. According to the context, Gemma is a state-of-the-art open model that represents the latest advancements in natural language processing (NLP). The model is designed to be easily adaptable and customizable for various applications, making it an attractive option for researchers and developers.


<span style='color:red'>2.4 With an embedding model</span> $e$ <span style='color:red'>and the word</span> $w$,<span style='color:red'> we obtain the embedded word</span> $e_w$<span style='color:red'>. Explain if the following affirmation is correct or not and why :</span>
$$e_{queen} - e_{mom} \approx e_{dad} - e_{king}$$

<span style='color:red'> Then find another example where the equation **works** and explain why.</span>

<span style='color:red'>2.5 The following graph shows the embedding of few words using the `word2vec` model. Explain, why it would be impossible to plot it, without changing the raw data. Then, how could it be done in the graph ?</span>

<p align="center"><img src="img-notebook/question_2_5_graph.png" alt="question_2_5_graph" width="400"/></p>

<span style='color:red'>2.6 In this graph, if I want to represent my data in 1-dimension, explain which axis would I keep and why ?</span>

<p align="center"><img src="img-notebook/question_2_6_graph.png" alt="question_2_6_graph" width="400"/></p>

<span style='color:red'>2.7 If I use a raw-LLM and simulate the following interraction (A = input to the model, B = answer of the model):</span>

* A : What is the distance between the earth and the moon ?
* B : Around 384,400 km
* A : And the sun ?

<span style='color:red'>What would be the answer ? And what a chatbot (i.e. ChatGPT) could do to fix the issue ?</span>

### Exercise 3 : Create a chat !

Using the Python module [Chainlit](https://github.com/Chainlit/chainlit), we can create an interactive web interface to discuss to our own chat-bot in just few lines of code.

The objective of this exercise is to create a chat where the user can upload a file so the bot can use it with RAG technique to update its data.

<span style='color:red'>3.1 Open the file `chat.py` and try to fill the missing parts by following the tasks bellow : </span>

* <span style='color:red'> Take a look at the `start()` function and try to understand the pipeline of the function `retrieval_qa_chain` (which functions are called, in which order, etc..). </span>
* <span style='color:red'> Fill the `#TODO 1` in the `process_pdf_file` function to load the PDF document using `PyPDFLoader`.</span>
* <span style='color:red'> Fill the `#TODO 2` in the `process_pdf_file` function to split the document into multiple smaller documents (chunking).</span>
* <span style='color:red'> Fill the `#TODO 3` in the `create_vector_store` function and transform the documents into vectors.</span>
* <span style='color:red'> Fill the `#TODO 4` in the `load_llm` function to load the model using `Ollama`.</span>
* <span style='color:red'> Fill the `#TODO 5` in the `retrieval_qa_chain` function to load the RAG prompt.</span>
* <span style='color:red'> Fill the `#TODO 6` in the `retrieval_qa_chain` function to create the chain Question/Answer. This function will call the other functions needed to create the chain (loading model and transforming document into vectors)</span>
* <span style='color:red'> Fill the `#TODO 7` in the `start` function and call the right function (make the call **asynchronous** !)<span>
* <span style='color:red'> Fill the `#TODO 8` in the `start` function to save the chain in the user session.<span>
* <span style='color:red'> Fill the `#TODO 9` in the `main` function and retrieve the chain from the user session.<span>
* <span style='color:red'> Fill the `#TODO 9` in the `main` function and retrieve the chain from the user session.<span>
* <span style='color:red'>Run the chat !</span>

*PS: To run the chat, use the commande below.*
```
chainlit run chat.py
```

<span style='color:red'>3.2 Upload a PDF document and try to ask the Chat bot some questions. Describe your findings (when does it work, when it doesn't, ...).</span>

It works good when you ask something that is written in the PDF document, if you ask something more deep or that is not in the field it is really slow to answer but works.

I used the laboratory instruction PDF for my findings.

If you ask something that does not make any sense like "test" he tries to answer for example for the promp "test" i had :

"I can answer the question based on the provided context. Here is my response:

The question being asked is related to a laboratory instruction on web mining, specifically regarding a chatbot called ChatGPT. ChatGPT is a large language model (LLM) that is capable of mimicking human conversations. The context provides information about the LLM and its capabilities, as well as instructions for completing an exercise related to the chatbot.

To answer the question, I can confirm that ChatGPT is a Large Language Model developed by OpenAI, and it is based on a template called chat.py. The model is designed to perform various NLP tasks, including conversational dialogue generation, and it has been shown to perform well on many benchmark tests [1].

I hope this information helps answer the question. If you have any further questions or need additional clarification, please let me know."

<span style='color:red'>3.3 Create your own Prompt (without using the `hub`) and use it in the chat bot pipeline. Explain your objective with it and describe your findings.</span>

Help : Here is the text used in `rlm/rag-prompt` :
```txt
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
```

I wanted the model to act just like he was before but to talk like he was Mario and it worked, here is an example :

![mario](mario.png "Mario")